In [1]:
import pickle
import matplotlib.pyplot as plt
import json
import networkx as nx
import pandas as pd
import numpy as np
from dwave.system import DWaveSampler, FixedEmbeddingComposite
import dwave.inspector

from plotters import plotHistogram, annotateSampleGraph, plotCSV, plotSampleGraph, plotArrangement
from qubo import createBQM
from sampleAnalysis import duplicates, findOverlaps, totalEntropy

identifier = "Haydn-3"
path = f"../Pickles/{identifier}/{identifier}_"

instruments = json.load(open(path + "instruments.json"))
phrases = pickle.load(open(path + "phrases.pkl", "rb"))
G = nx.read_graphml(path + "graph.graphml")
embedding = json.load(open(path + "embedding.json", "r"))
qa = FixedEmbeddingComposite(DWaveSampler(), embedding)

In [25]:
def recordAnnealTime(annealRange, bqm, filepath: str):
    
    file = pd.DataFrame()
    reads = 1000

    for a in annealRange:
        sampleset = qa.sample(bqm, num_reads=reads, annealing_time=a, label=f"{identifier}_at={a}")
        sample = sampleset.first

        A = annotateSampleGraph(sample.sample, G)

        new_row = pd.DataFrame({
            "Anneal time": [sampleset.info["timing"]["qpu_anneal_time_per_sample"]],
            "QPU time": [sampleset.info["timing"]["qpu_access_time"]],
            "Chain strength": [sampleset.info["embedding_context"]["chain_strength"]],
            "Total reads": [reads],
            "Chain break fraction": [sample.chain_break_fraction],
            "Lowest energy": [bqm.energy(sample.sample)],
            "Total entropy": [totalEntropy(A)],
            "Overlaps": [findOverlaps(annotateSampleGraph(sample.sample, G))],
            "Duplicates": [len(duplicates(sample.sample, G))]
            })
    
        print(f"Anneal time {a} us recorded!")
        file = pd.concat([file, new_row])

    file.to_csv(filepath, index=False, mode="a", header=False)

# 10–300 repeated four times
annealTimes = range(10,310,10)
bqm = createBQM(G, phrases, instruments, 6, 6, 1)
recordAnnealTime(annealTimes, bqm, path+"anneal-time.csv")

Anneal time 10 us recorded!
Anneal time 20 us recorded!
Anneal time 30 us recorded!
Anneal time 40 us recorded!
Anneal time 50 us recorded!
Anneal time 60 us recorded!
Anneal time 70 us recorded!
Anneal time 80 us recorded!
Anneal time 90 us recorded!
Anneal time 100 us recorded!
Anneal time 110 us recorded!
Anneal time 120 us recorded!
Anneal time 130 us recorded!
Anneal time 140 us recorded!
Anneal time 150 us recorded!
Anneal time 160 us recorded!
Anneal time 170 us recorded!
Anneal time 180 us recorded!
Anneal time 190 us recorded!
Anneal time 200 us recorded!
Anneal time 210 us recorded!
Anneal time 220 us recorded!
Anneal time 230 us recorded!
Anneal time 240 us recorded!
Anneal time 250 us recorded!
Anneal time 260 us recorded!
Anneal time 270 us recorded!
Anneal time 280 us recorded!
Anneal time 290 us recorded!
Anneal time 300 us recorded!


In [ ]:
plotCSV(path+"anneal-time.csv", "Anneal time", "Lowest energy")

# ~200us optimal???